# 🎌 Anime 4K Upscale v2 - ANTI-CRASH

**Tính năng mới: Auto-Split & Resume**
- 🛡️ Tự cắt phim thành các phần 15 phút
- 💾 Lưu từng phần vào Drive ngay khi xong
- 🔄 Bị crash? Chạy lại sẽ **tự làm tiếp** (không làm lại từ đầu)
- 🧩 Tự nối lại khi hoàn thành

**Cách dùng:**
1. Sửa `INPUT_DIR`
2. **Runtime → Run all** (`Ctrl+F9`)

In [ ]:
#@title ⚙️ CẤU HÌNH
INPUT_DIR = "/content/drive/MyDrive/Movie/Input" #@param {type:"string"}
OUTPUT_DIR = "/content/drive/MyDrive/Movie/Output" #@param {type:"string"}
MODE = "anime_4k" #@param ["anime_4k", "anime_fast", "ultra_fast"]

#@markdown ---
#@markdown ### Tùy chọn nâng cao
SEGMENT_TIME = 900 #@param {type:"integer"} # 15 phút
WORKER_ID = 1 #@param {type:"integer"}
TOTAL_WORKERS = 1 #@param {type:"integer"}

In [ ]:
#@title 🚀 CHẠY (Auto-Split)
import os, glob, subprocess, json, time, pickle, site, math, shutil
from datetime import timedelta

os.chdir('/content')

# Mount
print("📂 Mounting Drive...")
from google.colab import drive
drive.mount('/content/drive')

print("\n🖥️ GPU:")
!nvidia-smi --query-gpu=name,memory.free --format=csv

# ====== PATCH BASICSR (Running first) ======
print("\n🔧 Patching basicsr...")
patched = False
for sp in site.getsitepackages() + ['/usr/local/lib/python3.12/dist-packages']:
    pf = os.path.join(sp, 'basicsr/data/degradations.py')
    if os.path.exists(pf):
        with open(pf, 'r') as f: c = f.read()
        if 'functional_tensor' in c:
            c = c.replace('from torchvision.transforms.functional_tensor import rgb_to_grayscale', 'from torchvision.transforms.functional import rgb_to_grayscale')
            with open(pf, 'w') as f: f.write(c)
            print(f"   ✓ basicsr patched: {pf}")
            patched = True

if not patched:
    print("   ⚠️ Installing basicsr first...")
    !pip install basicsr -q
    # Retry patch
    for sp in site.getsitepackages():
        pf = os.path.join(sp, 'basicsr/data/degradations.py')
        if os.path.exists(pf):
            with open(pf, 'r') as f: c = f.read()
            c = c.replace('from torchvision.transforms.functional_tensor import rgb_to_grayscale', 'from torchvision.transforms.functional import rgb_to_grayscale')
            with open(pf, 'w') as f: f.write(c)
            print(f"   ✓ Patch retry success")

# ====== INSTALL ======
print("\n📦 Setup Real-ESRGAN...")
ESRGAN = '/content/Real-ESRGAN'
MODEL = f'{ESRGAN}/weights/realesr-animevideov3.pth'

if not os.path.exists(MODEL):
    !rm -rf {ESRGAN}
    !git clone https://github.com/xinntao/Real-ESRGAN.git
    os.chdir(ESRGAN)
    !pip install facexlib gfpgan -q
    !pip install -r requirements.txt -q
    !python setup.py develop 2>&1 | tail -1
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-animevideov3.pth -P weights/ -q

# ====== PATCH INFERENCE (Fix nb_frames) ======
inf_file = f'{ESRGAN}/inference_realesrgan_video.py'
if os.path.exists(inf_file):
    with open(inf_file, 'r') as f: code = f.read()
    if "ret['nb_frames'] = int(video_streams[0]" in code:
        new_block = '''    if 'nb_frames' in video_streams[0]:
        ret['nb_frames'] = int(video_streams[0]['nb_frames'])
    else:
        duration = float(ret.get('duration', 0))
        fps_parts = video_streams[0].get('r_frame_rate', '24/1').split('/')
        fps = float(fps_parts[0]) / float(fps_parts[1]) if len(fps_parts) == 2 else 24
        ret['nb_frames'] = int(duration * fps) if duration else 10000'''
        code = code.replace("ret['nb_frames'] = int(video_streams[0]['nb_frames'])", new_block)
        with open(inf_file, 'w') as f: f.write(code)
        print("   ✓ Inference script patched")

print("✅ Ready!")

# CONFIG
MODES = {
    'anime_4k': {'tile': 192, 'tile_pad': 24, 'crf': 22, 'preset': 'fast'},
    'anime_fast': {'tile': 128, 'tile_pad': 16, 'crf': 24, 'preset': 'veryfast'},
    'ultra_fast': {'tile': 64, 'tile_pad': 8, 'crf': 28, 'preset': 'ultrafast'}
}
cfg = MODES[MODE]

# HELPERS
def get_info(p):
    try:
        r = subprocess.run(['ffprobe','-v','quiet','-print_format','json','-show_format','-show_streams',p], capture_output=True, text=True, timeout=30)
        d = json.loads(r.stdout)
        v = next((s for s in d.get('streams',[]) if s.get('codec_type')=='video'), {})
        return {'w': int(v.get('width',0)), 'h': int(v.get('height',0)), 'dur': float(d.get('format',{}).get('duration',0))}
    except: return {'w':0,'h':0,'dur':0}

# SCAN
print(f"\n🔍 Scanning {INPUT_DIR}...")
all_files = []
if os.path.exists(INPUT_DIR):
    for root, dirs, files in os.walk(INPUT_DIR):
        for f in files: all_files.append(os.path.join(root, f))

VIDEO_EXT = ('.mp4', '.mkv', '.avi', '.mov', '.webm')
all_vids = sorted([f for f in all_files if f.lower().endswith(VIDEO_EXT)])
my_vids = [v for i,v in enumerate(all_vids) if (i % TOTAL_WORKERS) == (WORKER_ID - 1)]

to_process = []
for p in my_vids:
    name = os.path.basename(p)
    out = os.path.join(OUTPUT_DIR, name.rsplit('.',1)[0] + '_4K.mkv')
    info = get_info(p)
    
    if os.path.exists(out) and os.path.getsize(out) > info['dur']*100000: # Primitive check size
        print(f"⏭️ [DONE] {name}")
    elif info['w'] >= 3800:
        print(f"⏭️ [ALREADY 4K] {name}")
    elif info['w'] > 0:
        to_process.append({'path': p, 'name': name, 'info': info, 'out': out})
        print(f"✅ [QUEUE] {name} ({timedelta(seconds=int(info['dur']))})")

print(f"\n📊 {len(to_process)} videos to process")

# PROCESS LOOP
if to_process:
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    TEMP_DIR = '/content/segments'
    os.makedirs(TEMP_DIR, exist_ok=True)
    
    print(f"\n🚀 BẮT ĐẦU ANTI-CRASH PIPELINE (Mode: {MODE})")
    
    for idx, item in enumerate(to_process, 1):
        p, name, info, final_out = item['path'], item['name'], item['info'], item['out']
        print(f"\n{'='*60}")
        print(f"🎬 [{idx}/{len(to_process)}] {name}")
        print(f"   Output: {final_out}")
        
        try:
            # Calculate segments
            duration = info['dur']
            num_segs = math.ceil(duration / SEGMENT_TIME)
            if num_segs <= 1: num_segs = 1
            
            print(f"   🧩 Chia thành {num_segs} phần (mỗi phần {SEGMENT_TIME}s)")
            
            # Folder lưu các phần đã xong TRÊN DRIVE để resume
            CACHE_DIR = os.path.join(OUTPUT_DIR, ".cache", name)
            os.makedirs(CACHE_DIR, exist_ok=True)
            
            seg_files = []
            all_segs_done = True
            
            for i in range(num_segs):
                seg_name = f"part_{i:03d}.mkv"
                seg_path_drive = os.path.join(CACHE_DIR, seg_name)
                seg_files.append(seg_path_drive)
                
                # Check if done
                if os.path.exists(seg_path_drive) and os.path.getsize(seg_path_drive) > 1024*1024:
                    print(f"   ⏭️ Phần {i+1}/{num_segs} đã xong!")
                else:
                    all_segs_done = False
                    print(f"   ▶️ Đang xử lý phần {i+1}/{num_segs}...")
                    
                    # Extract segment
                    start_time = i * SEGMENT_TIME
                    raw_seg = f"{TEMP_DIR}/raw_{i}.mp4"
                    up_seg = f"{TEMP_DIR}/up_{i}.mp4"
                    
                    # Cut
                    cmd_cut = f'ffmpeg -y -ss {start_time} -i "{p}" -t {SEGMENT_TIME} -c copy -map 0 "{raw_seg}" -loglevel error'
                    os.system(cmd_cut)
                    
                    # Upscale
                    os.chdir(ESRGAN)
                    cmd_up = f'python inference_realesrgan_video.py -n realesr-animevideov3 -i "{raw_seg}" -o "{up_seg}" -s 2 --tile {cfg["tile"]} --tile_pad {cfg["tile_pad"]} --suffix "" --fp32'
                    os.system(cmd_up)
                    
                    if not os.path.exists(up_seg):
                        raise Exception(f"Upscale failed at segment {i+1}")
                        
                    # Encode & Move to Drive immediately
                    cmd_enc = f'ffmpeg -y -i "{up_seg}" -c:v libx265 -crf {cfg["crf"]} -preset {cfg["preset"]} -tag:v hvc1 -c:a copy -c:s copy "{seg_path_drive}" -loglevel error'
                    os.system(cmd_enc)
                    
                    # Cleanup temp
                    if os.path.exists(raw_seg): os.remove(raw_seg)
                    if os.path.exists(up_seg): os.remove(up_seg)
                    
                    print(f"      ✅ Xong phần {i+1}, đã lưu cache Drive")
            
            # Merge all segments
            if all_segs_done:
                print("   📦 Tất cả các phần đã có, đang nối lại...")
            else:
                print("   📦 Đã xử lý xong hết, đang nối lại...")
            
            list_file = f"{TEMP_DIR}/list.txt"
            with open(list_file, 'w') as f:
                for sf in seg_files:
                    f.write(f"file '{sf}'\n")
            
            # Concat
            cmd_concat = f'ffmpeg -y -f concat -safe 0 -i "{list_file}" -c copy -map_metadata 0 "{final_out}" -loglevel error'
            os.system(cmd_concat)
            
            # Cleanup Drive cache (Optional: uncomment to save space)
            # shutil.rmtree(CACHE_DIR)
            
            print(f"✅ HOÀN THÀNH VIDEO: {final_out}")

        except Exception as e:
            print(f"❌ Lỗi: {e}")
            print("💡 GỢI Ý: Chạy lại notebook, nó sẽ tự làm tiếp từ phần đang dở!")
            
else:
    print("✅ Không có video cần làm!")